### import the packages

In [ ]:
# Import os to easily join names of filepaths
import os

# Add the path of the PyOTIC Software to the system path
import sys
sys.path.append('../..') # notebook is located in original folder of PyOTIC Software 
#sys.path.append('/srv/ipython-repo/trunk') # notebook is located elsewhere (adjust this path to wher)
sys.path.append('../../../PyOTIC/pyoti/')
#Load investigator module
import pyoti
pyoti.info()
import pyotc

### create experiment and set data path

In [ ]:
# Choose the path, were the experiment should be created (or opened from)
#
# datadir: The path to where the experiment (and the data) are located
# datafile: The name of the file that contains the data. Here it is only used to generate dbfile.
#           The data is loaded further down upon creation of a Record.
# dbfile: The name of the database file the experiment is saved to (or loaded from).

datadir = '../exampleData/height_calibration_time_series/'
datafile = 'B01.bin'

# For the name of the experiment, exchange the extension '.bin' with '.fs'
dbfile = os.path.join(datadir, datafile.replace('.bin', '.fs'))

# Create the experiment (this will implicitly create/open the dbfile in datadir)
# and assign it to the variable 'experiment'
experiment = pyoti.open_experiment(dbfile)

cfgfile = '../../../PyOTIC/pyoti/pyoti/etc/record/ASWAD.cfg'

record = experiment.create_record(filename=datafile,  # data file to read
                                  cfgfile=cfgfile,  # setup specific configuration file
                                  directory=datadir)  # data directory

In [ ]:
experiment.add_view('height_calibration', 'alpha')

In [ ]:
dataregion = experiment.view('height_calibration')

motion = pyoti.create_motion(region=dataregion,  # region/view object
                             resolution=dataregion.samplingrate,  # sampling frequency 
                             impose_dwell_time=5.0,  # minimum length (in seconds) of the step
                             shift_time=0.1)  # delay (in seconds) after the defined step
# If you want to include the very first and the very last plateau, uncomment
#motion.include_first_last_plateau = True

# Show the autodetected plateaus, implicitly calls stepped.update()
motion.init_rfig(legend=True)

### determine amount of data (time) needed 

In [ ]:
N_avg = 40
ex_freq = 32.0

# posible minimum frequencies
print([ex_freq/ndiv for ndiv in range(1, 6)])

print('choose the minimum frequency:')
min_freq = float(input())

t_psd = 1/min_freq
print('Time for one psd: {0:1.3f} s'.format(t_psd))

t_total = N_avg * t_psd
print('Time needed for measurement: {0:1.3f} s'.format(t_total))

In [ ]:
import pyotc
## create height calibration from time series (i.e. from pyoti's motion object)
# height calib from motion object with specific excitation frequency
hcts = pyotc.HeightCalibTime(motion, ex_freq)

# set up experimental conditions
hcts.set_exp_conditions(29.2,  # temperature
                        1.0,  # radius
                        temp_err=0.001,
                        radius_err=0.01,
                        material='PS',
                        medium='water',
                        temp_unit='degC',
                        radius_unit='um')

hc = hcts.gen_height_calibration(N_avg, invert_z_signal=True, kws_psdfit={'ppd': 30})

In [ ]:
hc.setup_psd_fits(model='hydro')

bounds = {'psdX': (150, 8500), 'psdY': (150, 8500), 'psdZ': (150, 3000)}
#bounds = None
f_exclude = None

hc.fit_psds(bounds=bounds, f_exclude=f_exclude, plot_fits=True);

In [ ]:
hc.plot_pc_results(names=None)
#hc.save_hc_data('somename', directory=datadir)

In [ ]:
hc._names = ['psdX', 'psdY'] # trick to not fit z data

hc.focal_shift = 0.8
hc.ref_ind = 1.326
hc.set_wavelength(1064, unit='nm')

hc.fit_height_data(plot_fit=True, h0=-6)

In [ ]:
#hc.write_results_to_file()